# Learning the basics of algo trading with Oanda

In [1]:
# imports
import requests
import pandas as pd
from dotenv import load_dotenv
import os

In [2]:
# load environment variables
load_dotenv()

OANDA_API_KEY = os.getenv("OANDA_API_KEY")
OANDA_ACCT_NUM = os.getenv("OANDA_ACCT_NUM")
OANDA_URL = os.getenv("OANDA_URL")
SECURE_HEADER = {
    "Authorization": f"Bearer {OANDA_API_KEY}",
}

In [3]:
# create session
session = requests.Session()

In [4]:
# order setup
instrument = "EUR_USD"
count = 10
granularity = "H1"

In [5]:
url = f"{OANDA_URL}/v3/instruments/{instrument}/candles"
url

'https://api-fxpractice.oanda.com/v3/instruments/EUR_USD/candles'

In [6]:
params = dict(
    count=count,
    granularity=granularity,
    price="MBA",
)
params

{'count': 10, 'granularity': 'H1', 'price': 'MBA'}

In [7]:
# make request to API
response = session.get(url, params=params, headers=SECURE_HEADER)


In [8]:
response.json()

{'instrument': 'EUR_USD',
 'granularity': 'H1',
 'candles': [{'complete': True,
   'volume': 1467,
   'time': '2023-09-05T03:00:00.000000000Z',
   'bid': {'o': '1.07867', 'h': '1.07880', 'l': '1.07800', 'c': '1.07880'},
   'mid': {'o': '1.07875', 'h': '1.07888', 'l': '1.07808', 'c': '1.07888'},
   'ask': {'o': '1.07883', 'h': '1.07897', 'l': '1.07815', 'c': '1.07896'}},
  {'complete': True,
   'volume': 1623,
   'time': '2023-09-05T04:00:00.000000000Z',
   'bid': {'o': '1.07882', 'h': '1.07932', 'l': '1.07861', 'c': '1.07915'},
   'mid': {'o': '1.07889', 'h': '1.07940', 'l': '1.07868', 'c': '1.07923'},
   'ask': {'o': '1.07896', 'h': '1.07948', 'l': '1.07876', 'c': '1.07931'}},
  {'complete': True,
   'volume': 2263,
   'time': '2023-09-05T05:00:00.000000000Z',
   'bid': {'o': '1.07917', 'h': '1.07929', 'l': '1.07812', 'c': '1.07852'},
   'mid': {'o': '1.07925', 'h': '1.07936', 'l': '1.07820', 'c': '1.07860'},
   'ask': {'o': '1.07933', 'h': '1.07945', 'l': '1.07827', 'c': '1.07867'}},

In [9]:
data = response.json()
data.keys()

dict_keys(['instrument', 'granularity', 'candles'])

In [10]:
len(data["candles"])

10

In [13]:
prices = ['mid', 'bid', 'ask']
ohlc = ['o', 'h', 'l', 'c']

In [14]:
# format the data for the df
for price in prices:
    for oh in ohlc:
        print(f'{price}_{oh}')

mid_o
mid_h
mid_l
mid_c
bid_o
bid_h
bid_l
bid_c
ask_o
ask_h
ask_l
ask_c


In [17]:
our_data = []
for candle in data["candles"]:
    if candle["complete"] == False:
        continue
    new_dict = {}
    new_dict["time"] = candle["time"]
    new_dict["volume"] = candle["volume"]
    for price in prices:
        for oh in ohlc:
            new_dict[f"{price}_{oh}"] = candle[price][oh]
    our_data.append(new_dict)
our_data

[{'time': '2023-09-05T03:00:00.000000000Z',
  'volume': 1467,
  'mid_o': '1.07875',
  'mid_h': '1.07888',
  'mid_l': '1.07808',
  'mid_c': '1.07888',
  'bid_o': '1.07867',
  'bid_h': '1.07880',
  'bid_l': '1.07800',
  'bid_c': '1.07880',
  'ask_o': '1.07883',
  'ask_h': '1.07897',
  'ask_l': '1.07815',
  'ask_c': '1.07896'},
 {'time': '2023-09-05T04:00:00.000000000Z',
  'volume': 1623,
  'mid_o': '1.07889',
  'mid_h': '1.07940',
  'mid_l': '1.07868',
  'mid_c': '1.07923',
  'bid_o': '1.07882',
  'bid_h': '1.07932',
  'bid_l': '1.07861',
  'bid_c': '1.07915',
  'ask_o': '1.07896',
  'ask_h': '1.07948',
  'ask_l': '1.07876',
  'ask_c': '1.07931'},
 {'time': '2023-09-05T05:00:00.000000000Z',
  'volume': 2263,
  'mid_o': '1.07925',
  'mid_h': '1.07936',
  'mid_l': '1.07820',
  'mid_c': '1.07860',
  'bid_o': '1.07917',
  'bid_h': '1.07929',
  'bid_l': '1.07812',
  'bid_c': '1.07852',
  'ask_o': '1.07933',
  'ask_h': '1.07945',
  'ask_l': '1.07827',
  'ask_c': '1.07867'},
 {'time': '2023-09-

In [18]:
# turn into a dataframe
candles_df = pd.DataFrame(our_data)
candles_df

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2023-09-05T03:00:00.000000000Z,1467,1.07875,1.07888,1.07808,1.07888,1.07867,1.07880,1.07800,1.07880,1.07883,1.07897,1.07815,1.07896
1,2023-09-05T04:00:00.000000000Z,1623,1.07889,1.07940,1.07868,1.07923,1.07882,1.07932,1.07861,1.07915,1.07896,1.07948,1.07876,1.07931
2,2023-09-05T05:00:00.000000000Z,2263,1.07925,1.07936,1.07820,1.07860,1.07917,1.07929,1.07812,1.07852,1.07933,1.07945,1.07827,1.07867
3,2023-09-05T06:00:00.000000000Z,4522,1.07858,1.07864,1.07579,1.07666,1.07850,1.07857,1.07571,1.07658,1.07866,1.07872,1.07587,1.07674
4,2023-09-05T07:00:00.000000000Z,5181,1.07668,1.07719,1.07545,1.07565,1.07661,1.07711,1.07537,1.07557,1.07676,1.07727,1.07552,1.07573
5,2023-09-05T08:00:00.000000000Z,5164,1.07562,1.07588,1.07420,1.07483,1.07553,1.07581,1.07413,1.07475,1.07571,1.07596,1.07428,1.07491
6,2023-09-05T09:00:00.000000000Z,3503,1.07484,1.07571,1.07446,1.07484,1.07475,1.07564,1.07438,1.07477,1.07492,1.07579,1.07454,1.07492
7,2023-09-05T10:00:00.000000000Z,2854,1.07480,1.07494,1.07384,1.07394,1.07473,1.07486,1.07377,1.07387,1.07488,1.07501,1.07392,1.07402
8,2023-09-05T11:00:00.000000000Z,3353,1.07396,1.07398,1.07292,1.07326,1.07389,1.07391,1.07285,1.07318,1.07403,1.07406,1.07300,1.07333


In [19]:
# save df to pkl
candles_df.to_pickle("EUR_USD_H1.pkl")

In [20]:
# read back in
new_df = pd.read_pickle("EUR_USD_H1.pkl")
new_df

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2023-09-05T03:00:00.000000000Z,1467,1.07875,1.07888,1.07808,1.07888,1.07867,1.07880,1.07800,1.07880,1.07883,1.07897,1.07815,1.07896
1,2023-09-05T04:00:00.000000000Z,1623,1.07889,1.07940,1.07868,1.07923,1.07882,1.07932,1.07861,1.07915,1.07896,1.07948,1.07876,1.07931
2,2023-09-05T05:00:00.000000000Z,2263,1.07925,1.07936,1.07820,1.07860,1.07917,1.07929,1.07812,1.07852,1.07933,1.07945,1.07827,1.07867
3,2023-09-05T06:00:00.000000000Z,4522,1.07858,1.07864,1.07579,1.07666,1.07850,1.07857,1.07571,1.07658,1.07866,1.07872,1.07587,1.07674
4,2023-09-05T07:00:00.000000000Z,5181,1.07668,1.07719,1.07545,1.07565,1.07661,1.07711,1.07537,1.07557,1.07676,1.07727,1.07552,1.07573
5,2023-09-05T08:00:00.000000000Z,5164,1.07562,1.07588,1.07420,1.07483,1.07553,1.07581,1.07413,1.07475,1.07571,1.07596,1.07428,1.07491
6,2023-09-05T09:00:00.000000000Z,3503,1.07484,1.07571,1.07446,1.07484,1.07475,1.07564,1.07438,1.07477,1.07492,1.07579,1.07454,1.07492
7,2023-09-05T10:00:00.000000000Z,2854,1.07480,1.07494,1.07384,1.07394,1.07473,1.07486,1.07377,1.07387,1.07488,1.07501,1.07392,1.07402
8,2023-09-05T11:00:00.000000000Z,3353,1.07396,1.07398,1.07292,1.07326,1.07389,1.07391,1.07285,1.07318,1.07403,1.07406,1.07300,1.07333
